In [1]:
import pints
import pints.toy as toy
import numpy as np
import matplotlib.pyplot as plt
import GPy

import pints.emulator as emulator
import pints.emulator.utils as emutils
import pints.emulator.plot as emuplt

import copy
import operator

%matplotlib inline

In [2]:
# Create a model
model = pints.toy.FitzhughNagumoModel()

# Run a simulation
real_parameters = [0.1, 0.5, 3]

times = np.linspace(0, 20, 200)
org_values = model.simulate(real_parameters, times)

# take 5-10% of range as your std for noise
Vs, Rs = org_values.reshape(2, 200)
V_std = (Vs.max() - Vs.min()) * .1
R_std = (Rs.max() - Rs.min()) * .1

# Add noise
noise = [V_std, R_std]
values = org_values + np.random.normal(0, noise, org_values.shape)

# Create an object with links to the model and time series
problem = pints.MultiOutputProblem(model, times, values)

# Create a log-likelihood function (adds an extra parameter!)
real_log_likelihood = pints.KnownNoiseLogLikelihood(problem, noise)

In [3]:
bounds = pints.Boundaries(lower = [0, 0, 2], upper = [1, 1, 4])
log_prior = pints.UniformLogPrior(bounds)

In [4]:
input_parameters = log_prior.sample(500)
likelihoods = np.apply_along_axis(real_log_likelihood, 1, input_parameters)
likelihoods[:5]

array([-1073.31893811, -3633.51460843, -2842.15132316, -2369.01441131,
       -2421.94542317])

In [5]:
emu = emulator.GPEmulator(real_log_likelihood, input_parameters, 
                          likelihoods, 
                          normalize_input = True,
                         )

## Gradually trying bigger and bigger kernels

In [6]:
def is_prod_kernel(kernel):
    return type(kernel) == GPy.kern.src.prod.Prod

def is_add_kernel(kernel):
    return type(kernel) == GPy.kern.src.add.Add

def kernel_to_string(kernel, ident = 0):
    if kernel is None:
        return ""
    s = ""
    tab = ident * " "
    if is_prod_kernel(kernel) or is_add_kernel(kernel):
        op = "*" if is_prod_kernel(kernel) else "+"
        sub_kernels = []
        for sub_kernel in kernel.parameters:
            sub_kernels.append(kernel_to_string(sub_kernel, ident = ident + 1))
        s = "(" + op + "\n" + "\n".join(sub_kernels) + "\n" + tab + ")"
    else:
        # get name of kernel without "'>" characters
        name = str(type(kernel)).split(".")[-1]
        name = name[:-2]

        values = ",".join(["{:5f}".format(x) for x in kernel])
        s = name + "(" + values + ")"
    return " " * ident + s

def get_total_variance(kernel):
    ans = 0
    if is_prod_kernel(kernel) or is_add_kernel(kernel):
        for sub_kernel in kernel.parameters:
            ans += get_total_variance(sub_kernel)
    else:
        try:
            if hasattr(kernel, "variance"):
                ans += kernel.variance
            elif hasattr(kernel, "variances"):
                ans += kernel.variances
        except:
            # some kernels don't have variance as parameter
            ans += 0
            
    return ans
        
# function to get a measure of performance of kernel
# marginal_log_likelihood - alpha * var
def get_score(emu, alpha = 5):
    marg_log_likelihood = emu.get_log_marginal_likelihood()
    kern = emu.get_trained_kern()
    var = get_total_variance(kern)
    return marg_log_likelihood - alpha * var

In [7]:
n_parameters = emu.n_parameters()

base_kerns = [GPy.kern.RBF(n_parameters), 
              GPy.kern.RatQuad(n_parameters),
              GPy.kern.Linear(n_parameters),
              GPy.kern.PeriodicExponential(1),
              GPy.kern.PeriodicMatern52(1),
             ]

max_depth = 10
prev_kern = None
#time_limit?
#objective 
optimizer = 'lfgs'

In [8]:
#perform initial kernel selection
max_score = -1000
max_kern = None

for kern in base_kerns:
    emu.set_parameters(kernel = kern)
    emu.fit(normalizer = True, messages = False)
    score = get_score(emu)
    
    if score > max_score:
        print('here')
        max_score = score
        max_kern = emu.get_trained_kern()
        
    print(emu.get_trained_kern())
    print('Marginal log likelihood - alpha * var: ', score)

emu.set_parameters(kernel = max_kern)
emu.fit(normalizer = True)
emu.get_gp()

here
  rbf.         |               value  |  constraints  |  priors
  variance     |  0.4995068677375631  |      +ve      |        
  lengthscale  |  0.6797206343919966  |      +ve      |        
Marginal log likelihood - alpha * var:  [-125.56366524]
here
  RatQuad.     |               value  |  constraints  |  priors
  variance     |    1.66431724152907  |      +ve      |        
  lengthscale  |  0.4743807643693985  |      +ve      |        
  power        |   0.144694383226453  |      +ve      |        
Marginal log likelihood - alpha * var:  [-106.28290583]
  linear.    |               value  |  constraints  |  priors
  variances  |  0.1943399333289162  |      +ve      |        
Marginal log likelihood - alpha * var:  [-491.18789303]


 /anaconda3/lib/python3.6/site-packages/GPy/core/gp.py:87: UserWarning:Your kernel has a different input dimension 1 then the given X dimension 3. Be very sure this is what you want and you have not forgotten to set the right input dimenion in your kernel


  periodic_exponential.  |               value  |  constraints  |  priors
  variance               |  7.0743645411186895  |      +ve      |        
  lengthscale            |  17.147405685748947  |      +ve      |        
  period                 |   6.352038962705615  |      +ve      |        
Marginal log likelihood - alpha * var:  [-563.40473541]
  periodic_Matern52.  |               value  |  constraints  |  priors
  variance            |   18.71416523246459  |      +ve      |        
  lengthscale         |  3.2364500402034344  |      +ve      |        
  period              |   5.781161713255169  |      +ve      |        
Marginal log likelihood - alpha * var:  [-618.8898742]
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s04  0002   4.029069e+02   2.170879e+04 
    00s17  0008   1.049281e+02   6.800572e+02 
    00s19  0009   9.812666e+01   5.175732e+01 
    00s35  0018   9.796132e+01   4.931623e-07 
Runtime:     00s35
Optimizati

GP_regression.,value,constraints,priors
RatQuad.variance,1.6612898865203793,+ve,
RatQuad.lengthscale,0.4744773822643596,+ve,
RatQuad.power,0.14500841790406938,+ve,
Gaussian_noise.variance,0.028244283224871738,+ve,


In [9]:
depth = 1
prev_max_kern = max_kern
while depth < max_depth:
    print("-" * 10 + "Depth " + str(depth) + '-'*10)
    
    max_score = -1000
    for op in [operator.add, operator.mul]:
        for kern in base_kerns:
            current_kern = op(prev_max_kern, kern)
            emu.set_parameters(kernel = current_kern)
            emu.fit(normalizer = True, messages = False)
            score = get_score(emu)
    
            if score > max_score:
                max_kern = emu.get_trained_kern()
                max_score = score
            
    prev_max_kern = max_kern
    print(max_kern)
    print(max_score)
    
    depth += 1

----------Depth 1----------
  sum.                 |                value  |  constraints  |  priors
  RatQuad.variance     |  0.19040567860439478  |      +ve      |        
  RatQuad.lengthscale  |  0.42022418346424667  |      +ve      |        
  RatQuad.power        |   0.7775446542031264  |      +ve      |        
  rbf.variance         |   0.5697405353152224  |      +ve      |        
  rbf.lengthscale      |   1.1537920751913022  |      +ve      |        
[-97.74605625]
----------Depth 2----------
  sum.                 |                value  |  constraints  |  priors
  RatQuad.variance     |  0.11113629169310475  |      +ve      |        
  RatQuad.lengthscale  |   0.4559298600528887  |      +ve      |        
  RatQuad.power        |   1.4306636754768813  |      +ve      |        
  rbf.variance         |  0.42689651676123425  |      +ve      |        
  rbf.lengthscale      |    0.963723059734677  |      +ve      |        
  linear.variances     |  0.04251892175485914  |     

 /anaconda3/lib/python3.6/site-packages/GPy/kern/src/periodic.py:40: RuntimeWarning:overflow encountered in cos
 /anaconda3/lib/python3.6/site-packages/GPy/kern/src/periodic.py:40: RuntimeWarning:invalid value encountered in cos


  sum.                 |                 value  |  constraints  |  priors
  RatQuad.variance     |   0.11110849978895496  |      +ve      |        
  RatQuad.lengthscale  |      0.45598227385762  |      +ve      |        
  RatQuad.power        |    1.4312747611342196  |      +ve      |        
  rbf.variance         |    0.4269287341591746  |      +ve      |        
  rbf.lengthscale      |    0.9637102936602474  |      +ve      |        
  linear.variances     |   0.01470716804444929  |      +ve      |        
  linear_1.variances   |  0.013804258164515906  |      +ve      |        
  linear_2.variances   |  0.013996979179333414  |      +ve      |        
[-96.41983675]
----------Depth 5----------
  sum.                              |                 value  |  constraints  |  priors
  RatQuad.variance                  |   0.11111684176684784  |      +ve      |        
  RatQuad.lengthscale               |   0.45597911459686236  |      +ve      |        
  RatQuad.power               

 /anaconda3/lib/python3.6/site-packages/GPy/kern/src/periodic.py:100: RuntimeWarning:overflow encountered in true_divide


  sum.                              |                   value  |  constraints  |  priors
  RatQuad.variance                  |    0.020045725208130163  |      +ve      |        
  RatQuad.lengthscale               |      26.126351078876823  |      +ve      |        
  RatQuad.power                     |      13913.944521420417  |      +ve      |        
  rbf.variance                      |     0.06598963369759965  |      +ve      |        
  rbf.lengthscale                   |     0.42276493651771135  |      +ve      |        
  linear.variances                  |   0.0042509223981147025  |      +ve      |        
  linear_1.variances                |   6.680081506945678e-05  |      +ve      |        
  linear_2.variances                |    0.037176105685885474  |      +ve      |        
  periodic_exponential.variance     |  4.8576158538090956e-08  |      +ve      |        
  periodic_exponential.lengthscale  |      25.970374993027335  |      +ve      |        
  periodic_exponentia

 /anaconda3/lib/python3.6/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


  sum.                              |                   value  |  constraints  |  priors
  RatQuad.variance                  |    0.020030328610323105  |      +ve      |        
  RatQuad.lengthscale               |        26.1254134800574  |      +ve      |        
  RatQuad.power                     |      13913.944536789184  |      +ve      |        
  rbf.variance                      |     0.06597718190860712  |      +ve      |        
  rbf.lengthscale                   |     0.42266000054337405  |      +ve      |        
  linear.variances                  |    0.039913133807537846  |      +ve      |        
  linear_1.variances                |   7.019436785813107e-05  |      +ve      |        
  linear_2.variances                |   0.0015366045036164372  |      +ve      |        
  periodic_exponential.variance     |    4.85742556558384e-08  |      +ve      |        
  periodic_exponential.lengthscale  |      25.970376497660887  |      +ve      |        
  periodic_exponentia

In [10]:
get_total_variance(prev_max_kern)

array([0.58341252])